# MNIST with Keras

## Import Data

In [1]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


- **x_train, x_test**: array of grayscale image data, shape (num_samples, 28, 28)
- **y_train, y_test**: array of digit labels int[0..9] shape (num_samples)

In [2]:
n_class = 10
batch_size = 600
epoch_limit = 100

## Preprocessing

Reshape x from (28,28) to (764,)

In [3]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])

Convert training and testing data to float and divide by 255 to make all values range between [0..1]

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

One-hot encoding for training and testing labels based on number of possible classes.

Example: 5 => (0, 0, 0, 0, 0, 1, 0, 0, 0, 0)

In [5]:
import keras

y_train = keras.utils.to_categorical(y_train, n_class)
y_test = keras.utils.to_categorical(y_test, n_class)

## Model

Use `Sequential` model from Keras to create multi-layer perceptron

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K

model = Sequential()
model.add(Dense(196,
                activation='relu',
                kernel_initializer='he_uniform',
                input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(196,
                activation='relu',
                kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(98,
                activation='relu',
                kernel_initializer='he_uniform'))
model.add(Dropout(0.1))
model.add(Dense(n_class,
                activation='softmax',
                kernel_initializer='glorot_normal'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 196)               153860    
_________________________________________________________________
dropout_1 (Dropout)          (None, 196)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 196)               38612     
_________________________________________________________________
dropout_2 (Dropout)          (None, 196)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 98)                19306     
_________________________________________________________________
dropout_3 (Dropout)          (None, 98)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                990       
Total para

Weight initialization algorithm is determined by `kernel_initializer` param.
- `he_uniform` for layers with `relu` activation
- `glorot_normal` or _xavier_ for layers with `softmax` activation

This is chosen because on the research paper, _he_ focuses on relu and _xavier glorot_ focuses on sigmoid and softmax.

## Optimizers

Tested with two optimizers, adam and sgd, with adam winning by 1% increase in accuracy.

In [7]:
from keras import optimizers

adam = optimizers.Adam(lr=0.001,
                       beta_1=0.9,
                       beta_2=0.999,
                       epsilon=None,
                       decay=0.00001,
                       amsgrad=False)

sgd = optimizers.SGD(lr=0.001,
                     momentum=0.001,
                     decay=0.00001,
                     nesterov=False)

A higher value of learning rate (e.g. 0.1) will result in the model failing to converge.

## Training

In [8]:
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epoch_limit,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 6s 103us/step - loss: 0.6240 - acc: 0.8045 - val_loss: 0.1981 - val_acc: 0.9406
Epoch 2/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.2194 - acc: 0.9354 - val_loss: 0.1305 - val_acc: 0.9600
Epoch 3/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.1580 - acc: 0.9521 - val_loss: 0.1079 - val_acc: 0.9658
Epoch 4/100
60000/60000 [==============================] - 3s 44us/step - loss: 0.1281 - acc: 0.9617 - val_loss: 0.0903 - val_acc: 0.9713
Epoch 5/100
60000/60000 [==============================] - 3s 42us/step - loss: 0.1046 - acc: 0.9678 - val_loss: 0.0839 - val_acc: 0.9735
Epoch 6/100
60000/60000 [==============================] - 3s 45us/step - loss: 0.0912 - acc: 0.9723 - val_loss: 0.0824 - val_acc: 0.9737
Epoch 7/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.0819 - acc: 0.9746 - val_loss: 0.0756 -

## Testing Scores

In [9]:
score = model.evaluate(x_test, y_test,
                       verbose=1)

print('Test Loss = {}, Test Accuracy = {}'.format(score[0], score[1]))

10000/10000 [==============================] - 1s 55us/step
Test Loss = 0.08469229024424053, Test Accuracy = 0.9822
